In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
from Bio.KEGG import Enzyme
import matplotlib
import io
import time
import pickle
import re
from rich.progress import Progress
from rdkit import Chem
from collections import Counter
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdFMCS
import pubchempy as pcp
from bs4 import BeautifulSoup
from urllib.parse import quote
import json

import math
import os

from ast import literal_eval

The goal here needs to be assign ownership of a ligand to a domain. So need to have a function that iterates through each ligand (bound ligand ID). Having the ligand entity id and pdb entity id are potentially unnecessary in the cypher for this query.

Can groupby ligand entity id to get all of the contacts for the ligand.

Should first implement a function that assigns ownership in terms of orginal procognate paper i.e. if it owns more than 70% of contacts, it is the ligand owner.

Definition of a ligand needs to be clear: it is the instance of an entity, not just the entity.

A plot of ligand ownership would be good. 

these are the possible groups we need to create hierarchies for.

VDW clashes: disregard
Covalent: Is prioritised.
Ionic bond > Halogen bond > H-bond > Polar > weak H-bond > weak polar > vdw - so ,if all of the bonds are in this list, apply a numeric value and select the top one.

We rank atom-atom and atom-plane interactions independently.

Instead just going with the existence of a contact, i.e. no filtering or anything complex for now

In [146]:
def assign_ownership_percentile(ligands_df, percentile_cut_off = 0.7):
    ligands_df["total_counts"] = ligands_df.groupby("bound_ligand_id").transform("size")
    ligands_df["domain_counts"] = ligands_df.groupby(["bound_ligand_id", "cath_domain"]).transform("size")
    ligands_df["domain_ownership"] = np.where(ligands_df.domain_counts > (ligands_df.total_counts * 0.7), ligands_df["cath_domain"], np.nan)
    ligands_df.loc[ligands_df.groupby("bound_ligand_id")["domain_ownership"].transform(lambda x: x.isnull().all()), "domain_ownership"] = "shared"
    return ligands_df

def assign_ownership_percentile_categories(ligands_df, domain_grouping_key = "cath_domain"):
    ligands_df["total_contact_counts"] = ligands_df.groupby("bound_ligand_id").transform("size")
    ligands_df["domain_contact_counts"] = ligands_df.groupby(["bound_ligand_id", domain_grouping_key]).transform("size")
    hbond_counts = ligands_df.explode('contact_type').groupby(['bound_ligand_id', domain_grouping_key])['contact_type'].apply(lambda x: (x == 'hbond').sum()).rename("domain_hbond_counts").reset_index()
    ligands_df = ligands_df.merge(hbond_counts, how = "left", on = ["bound_ligand_id", domain_grouping_key], indicator = True)
    assert(len(ligands_df.loc[ligands_df._merge != "both"]) == 0)
    ligands_df.drop(columns = ["_merge"], inplace = True)
    ligands_df["domain_hbond_perc"] = ligands_df.domain_hbond_counts / ligands_df.total_contact_counts
    ligands_df["domain_contact_perc"] = ligands_df.domain_contact_counts / ligands_df.total_contact_counts
    ligands_df["domain_ownership"] = np.where(
        ligands_df["domain_contact_perc"] == 1, "uniquely_binding_domain",
        np.where(
            ligands_df["domain_contact_perc"] >= 0.7, "dominant_binding_domain",
            np.where(
                (ligands_df["domain_contact_perc"] >= 0.3)
                & (ligands_df["domain_contact_perc"] < 0.7), "partner_binding_domain",
                np.where(ligands_df["domain_contact_perc"] < 0.3, "minor_binding_domain", np.nan)
            )
        )
    )
    return ligands_df

In [147]:
cath_residue_df = pd.read_csv("pdbe_graph_files/cath_pdb_residue_interactions_distinct.csv", na_values = ["NaN", "None"], keep_default_na = False)
cath_residue_df = cath_residue_df.loc[cath_residue_df.bound_ligand_name != "UNL"]
cath_residue_df["contact_type"] = cath_residue_df["contact_type"].apply(literal_eval)

cath_residue_df_domains = assign_ownership_percentile_categories(cath_residue_df.copy(), "cath_domain")

In [149]:
cath_domains = cath_residue_df_domains[["pdb_id", "protein_entity_id", "protein_entity_ec", "cath_domain", "cath_class", "cath_architecture", "cath_topology","cath_homology", "cath_name", "ligand_entity_id", "bound_ligand_id", "bound_ligand_name", "total_contact_counts", "domain_contact_counts", "domain_hbond_counts", "domain_contact_perc", "domain_hbond_perc", "domain_ownership"]].copy()

In [180]:
cath_residue_df_domains.loc[cath_residue_df_domains.bound_ligand_id == "9nse_6_O_1"].bound_ligand_auth_id

3728735    801
3728736    801
3728737    801
3728738    801
3728739    801
3728740    801
3728781    801
3728782    801
3728783    801
3728784    801
3728810    801
3728811    801
Name: bound_ligand_auth_id, dtype: int64

In [150]:
cath_domains.drop_duplicates(inplace = True)

In [151]:
cath_domains.to_csv("domain_ownership/cath_domain_ownership.csv")

In [167]:


scop_domains_info = pd.read_csv("domain_ownership/dir.cla.scop.1_75.txt", sep = "\t", comment = "#", header = None, names = ["scop_id", "pdb_id", "description", "sccs", "domain_sunid", "ancestor_sunid"])
scop_id_levels = ["cl_id", "cf_id", "sf_id", "fa_id", "dm_id", "sp_id", "px_id"]
scop_domains_info[scop_id_levels] = scop_domains_info.ancestor_sunid.str.split(",", expand = True)

scop_descriptions = pd.read_csv("domain_ownership/dir.des.scop.1_75.txt", sep = "\t", comment = "#" , header = None, names = ["level_sunid", "level", "level_sccs", "level_sid", "level_description"])

def clean_and_merge_scop_col(df, column_id, description_df):
    level = df[column_id].str.split("=").str.get(0).values[0]
    df[column_id] = df[column_id].str.split("=").str.get(1).astype(int)
    df = df.merge(description_df.loc[description_df.level == level, ["level_sunid", "level", "level_description"]],left_on = column_id, right_on = "level_sunid", indicator = True)
    df.rename(columns = {"level_description": f"{level}_description"}, inplace = True)
    assert len(df.loc[df._merge != "both"]) == 0
    df.drop(columns = ["_merge", "level_sunid", "level"], inplace = True)
    return df

def complete_unmatched_domains(df, class_codes, fold_codes, superfamily_codes):
    df = df.merge(class_codes, left_on = "scop_class_id", right_on = "cl_id", how = "left", indicator = True)
    df["cl_description_x"] = df["cl_description_x"].fillna(df["cl_description_y"])
    df["cl_id_x"] = df["cl_id_x"].fillna(df["scop_class_id"])
    df.rename(columns = {"cl_id_x" : "cl_id", "cl_description_x": "cl_description"}, inplace = True)
    df.drop(columns = ["_merge", "cl_description_y", "cl_id_y"], inplace = True)
    df = df.merge(fold_codes, left_on = "scop_fold_id", right_on = "cf_id", how = "left", indicator = True)
    df["cf_description_x"] = df["cf_description_x"].fillna(df["cf_description_y"])
    df["cf_id_x"] = df["cf_id_x"].fillna(df["scop_fold_id"])
    df.rename(columns = {"cf_id_x" : "cf_id", "cf_description_x": "cf_description"}, inplace = True)
    df.drop(columns = [ "_merge", "cf_description_y", "cf_id_y"], inplace = True)
    df = df.merge(superfamily_codes, left_on = "scop_superfamily_id", right_on = "sf_id", how = "left", indicator = True)
    df["sf_description_x"] = df["sf_description_x"].fillna(df["sf_description_y"])
    df["sf_id_x"] = df["sf_id_x"].fillna(df["scop_superfamily_id"])
    df.rename(columns = {"sf_id_x" : "sf_id", "sf_description_x": "sf_description"}, inplace = True)
    df.drop(columns = ["_merge", "sf_description_y", "sf_id_y"], inplace = True)
    return df

for column in scop_id_levels:
    scop_domains_info = clean_and_merge_scop_col(scop_domains_info, column, scop_descriptions)
    
scop_domains_info.drop(columns = ["pdb_id"], inplace = True)

class_codes = scop_domains_info[["cl_id", "cl_description"]].drop_duplicates()
fold_codes = scop_domains_info[["cf_id", "cf_description"]].drop_duplicates()
superfamily_codes = scop_domains_info[["sf_id", "sf_description"]].drop_duplicates()

In [168]:
scop_residue_df = pd.read_csv("pdbe_graph_files/scop_pdb_residue_interactions_distinct.csv", na_values = ["NaN", "None"], keep_default_na = False)
scop_residue_df = scop_residue_df.loc[scop_residue_df.bound_ligand_name != "UNL"]
scop_residue_df["contact_type"] = scop_residue_df["contact_type"].apply(literal_eval)
scop_residue_df_domains = assign_ownership_percentile_categories(scop_residue_df.copy(), "scop_id")

scop_domains = scop_residue_df_domains[["pdb_id", "protein_entity_id", "protein_entity_ec", "scop_sunid","scop_description", "scop_sccs", "scop_class_id", "scop_fold_id", "scop_superfamily_id", "scop_id", "ligand_entity_id", "bound_ligand_id", "bound_ligand_name", "total_contact_counts", "domain_contact_counts", "domain_hbond_counts", "domain_contact_perc", "domain_hbond_perc", "domain_ownership"]].copy()
scop_domains.drop_duplicates(inplace = True)

scop_domains = scop_domains.merge(scop_domains_info, how = "left", on = "scop_id", indicator = True)

scop_domains_matched = scop_domains.loc[scop_domains._merge == "both"].copy().drop(columns = ["_merge"])
scop_domains_unmatched = scop_domains.loc[scop_domains._merge != "both"].copy().drop(columns = ["_merge"])

scop_domains_unmatched = complete_unmatched_domains(scop_domains_unmatched, class_codes, fold_codes, superfamily_codes)
scop_domains = pd.concat([scop_domains_matched, scop_domains_unmatched])

In [170]:
scop_domains.to_csv("domain_ownership/scop_domain_ownership.csv")

In [ ]:
#residue_df["contact_type_list"] = residue_df["contact_type"]
# residue_df["len"] = residue_df.contact_type.str.len()
# residue_df = residue_df.explode("contact_type").reset_index()
#residue_df.loc[residue_df.interaction_type == "atom-atom", "contact_value"] = residue_df.loc[residue_df.interaction_type == "atom-atom", "contact_type"].apply(lambda x: hierarchy.index(x) if x in hierarchy else -1)
# residue_df = residue_df.loc[residue_df["contact_type"] != "vdw_clash"]
# residue_df.groupby("contact_type").contact_type_list.value_counts().to_csv("contact_types_list.csv")